In [1]:
import pymysql
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
cnx = pymysql.connect(
    user="multi",
    password="*****!",
    host="ec2-15-152-211-160.ap-northeast-3.compute.amazonaws.com",
    database="Data_Mart",
)

query = "SELECT * FROM credit_analysis_model_a;"
model_a = pd.read_sql(query, cnx) 

query = "SELECT * FROM credit_analysis_model_b;"
model_b = pd.read_sql(query, cnx) 

cnx.close()

c:\Users\sjpar\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\sjpar\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


### 극단값/이상치 제거: model.describe() 봐서 판단하면서 제거
1. 컬럼별 극단값 0.001% 제거할 예정 (상위 3개 하위 3개) - 하지만 제거 안해도 될 값은 안 하는 게 좋음 
2. 비율 볼 때 극단치 제거가 목표!! 예: ebitda_margin == -12526.985 
3. 일반 계정은 제거 안 하는 게 좋음. 예: 매출액 / 매출원가 / EBITDA 등등 -> 제거하면 삼성전자 / SK / 현대자동차 같은 대기업 당연하게 제거됨 
   - 비율만 제거! 
4. 경제 지표 / 기업 리뷰 데이터 제거 안 함 
5. 하위 값 0.001% 음수인 경우만 제거 - 양수면 제거 안 함 (0에 제일 가까운 값 제거 할 필요 X)

In [3]:
model_a.describe()

,ebitda_margin,ebitda_to_interest_expense,debt_ratio,dependence_on_net_borrowings,net_borrowings_to_ebitda,revenue,cogs,selling_general_administrative_expenses,ebit,ebit_margin,ebitda_to_sales_revenue,total_assets,return_on_assets,ebitda,financial_expenses,corporate_tax,operating_cash_flow,free_cash_flow,total_liabilities,total_equity,total_borrowings,net_borrowings,borrowing_dependency,total_borrowings_to_ebitda,debt_to_net_income_ratio,total_assets_leverage,current_liabilities,working_capital,current_liabilities_ratio,quick_assets,quick_ratio,cash_and_cash_equivalents,short_term_borrowings,days_sales_outstanding,average_accounts_receivable_per_sales_turnover,market_capitalization,minimum_wag,uskor_exchange_average,PPI_year,kor_usa_ir_diff,kr_standard_yield,count,rating,paywellfare,worklifebal,culture,opportunity,manager,recommend,ceo,potential,crb_index_avg
count,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3.097000e+03,3.097000e+03,3.097000e+03,3.097000e+03,3097.000000,3097.000000,3.097000e+03,3097.000000,3.097000e+03,3.097000e+03,3.097000e+03,3.097000e+03,3.097000e+03,3.097000e+03,3.097000e+03,3.097000e+03,3.097000e+03,3097.000000,3097.000000,3097.000000,3097.000000,3.097000e+03,3.097000e+03,3097.000000,3.097000e+03,3097.000000,3.097000e+03,3.097000e+03,3097.000000,3097.000000,3.097000e+03,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000
mean,-2.882557,40.948553,0.478166,0.308153,1.410622,3.603240e+12,2.803462e+12,4.231992e+11,2.152122e+11,-3.005391,-0.028826,5.300108e+12,1.342577,2.211398e+11,1.076804e+11,6.243961e+10,3.438505e+11,6.697211e+11,2.825123e+12,2.473725e+12,6.446069e+11,2.736895e+11,0.308153,5.116712,21.746514,2.929656,1.428790e+12,5.251109e+11,0.851399,1.524762e+12,1.531084,3.709174e+11,3.166636e+11,64.683186,0.177214,2.760508e+12,8486.664514,1178.087317,107.789254,0.047410,1.551017,694.362344,2.825131,2.784634,3.006500,2.594659,2.679684,2.239396,1.290491,1.316422,1.187091,214.687310
std,227.773388,760.817358,0.204890,14.923190,165.902521,1.427841e+13,1.056336e+13,2.586860e+12,2.128457e+12,227.742018,2.277734,2.351324e+13,9.807262,2.137042e+12,6.517140e+11,4.743611e+11,2.770221e+12,4.756917e+12,1.267032e+13,1.339906e+13,2.758428e+12,2.200212e+12,14.923190,164.212715,366.579936,27.814039,5.243446e+12,5.242615e+12,0.816507,7.790042e+12,2.213052,1.782076e+12,1.276890e+12,75.205910,0.206044,2.299328e+13,531.520977,63.759380,6.102122,0.398899,0.951537,1305.655585,0.503111,0.619845,0.592226,0.479991,0.402268,0.422921,0.690566,0.667644,0.691573,50.326558
min,-12526.985234,-1312.671136,0.014647,-532.783154,-5367.015660,1.747813e+08,0.000000e+00,5.417029e+06,-3.384362e+13,-12526.985234,-125.269852,1.247618e+08,-164.393583,-3.384362e+13,-7.323143e+11,-4.695683e+11,-2.347750e+13,-1.360712e+13,7.506294e+07,-7.081860e+11,0.000000e+00,-4.097272e+13,-532.783154,-5787.109634,-8162.082210,-689.306717,1.752545e+08,-1.476803e+13,0.025778,7.245676e+07,0.042676,4.183128e+06,0.000000e+00,0.000000,0.000000,0.000000e+00,7530.000000,1100.580000,103.030000,-0.560000,0.500000,1.000000,0.060000,0.030000,0.030000,0.090000,0.090000,0.060000,0.060000,0.060000,0.060000,157.422309
25%,0.188978,-0.050986,0.321539,0.046023,-1.448183,2.266950e+11,1.456726e+11,2.155040e+10,6.506744e+08,0.188277,0.001890,3.125910e+11,-0.196224,6.551495e+08,2.201068e+09,7.307400e+08,2.826514e+09,8.615194e+09,1.125769e+11,1.552552e+11,1.286919e+10,-2.420955e+10,0.046023,0.000000,-1.037432,1.468990,8.170339e+10,-6.548095e+09,0.487719,9.457295e+10,0.647453,1.771952e+10,2.817016e+09,39.881978,0.109266,1.133564e+11,8350.000000,1144.610000,103.480000,-0.290000,1.000000,129.000000,2.500000,2.350000,2.640000,2.260000,2.440000,1.970000,0.750000,0.810000,0.660000,187.793544
50%,3.723956,2.227229,0.487419,0.221925,0.113407,6.383948e+11,4.672618e+11,6.057857e+10,2.063504e+10,3.721099,0.037240,7.641384e+11,2.304863,2.0697

In [4]:
model_b.describe()

,ebitda_margin,ebitda_to_interest_expense,debt_ratio,dependence_on_net_borrowings,net_borrowings_to_ebitda,revenue,cogs,selling_general_administrative_expenses,ebit,ebit_margin,ebitda_to_sales_revenue,total_assets,return_on_assets,ebitda,financial_expenses,corporate_tax,operating_cash_flow,free_cash_flow,total_liabilities,total_equity,total_borrowings,net_borrowings,borrowing_dependency,total_borrowings_to_ebitda,debt_to_net_income_ratio,total_assets_leverage,current_liabilities,working_capital,current_liabilities_ratio,quick_assets,quick_ratio,cash_and_cash_equivalents,short_term_borrowings,days_sales_outstanding,average_accounts_receivable_per_sales_turnover,market_capitalization,minimum_wag,uskor_exchange_average,PPI_year,kor_usa_ir_diff,kr_standard_yield,count,rating,paywellfare,worklifebal,culture,opportunity,manager,recommend,ceo,potential,crb_index_avg,gross_profit_margin,operating_profit_margin,net_profit_margin,return_on_equity,return_on_invested_capital,current_ratio,non_current_debt_ratio,equity_ratio,interest_coverage_ratio,debt_to_equity_ratio,net_debt_ratio,retention_ratio,earnings_per_share,book_value_per_share,price_earnings_ratio,price_to_book_ratio,price_cash_flow_ratio,enterprise_value_to_ebitda,operating_income,net_income,fixed_assets,total_asset_turnover,net_working_capital_turnover,fixed_asset_turnover,accounts_receivable_turnover,inventory_turnover,accounts_payable_turnover
count,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3.097000e+03,3.097000e+03,3.097000e+03,3.097000e+03,3097.000000,3097.000000,3.097000e+03,3097.000000,3.097000e+03,3.097000e+03,3.097000e+03,3.097000e+03,3.097000e+03,3.097000e+03,3.097000e+03,3.097000e+03,3.097000e+03,3097.000000,3097.000000,3097.000000,3097.000000,3.097000e+03,3.097000e+03,3097.000000,3.097000e+03,3097.000000,3.097000e+03,3.097000e+03,3097.000000,3097.000000,3.097000e+03,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3.097000e+03,3.097000e+03,3097.000000,3097.000000,3097.000000,3097.000000,3.097000e+03,3.097000e+03,3.097000e+03,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000,3097.000000
mean,-2.882557,40.948553,0.478166,0.308153,1.410622,3.603240e+12,2.803462e+12,4.231992e+11,2.152122e+11,-3.005391,-0.028826,5.300108e+12,1.342577,2.211398e+11,1.076804e+11,6.243961e+10,3.438505e+11,6.697211e+11,2.825123e+12,2.473725e+12,6.446069e+11,2.736895e+11,0.308153,5.116712,21.746514,2.929656,1.428790e+12,5.251109e+11,0.851399,1.524762e+12,1.531084,3.709174e+11,3.166636e+11,64.683186,0.177214,2.760508e+12,8486.664514,1178.087317,107.789254,0.047410,1.551017,694.362344,2.825131,2.784634,3.006500,2.594659,2.679684,2.239396,1.290491,1.316422,1.187091,214.687310,22.889080,4.839828,-3.677295,4.555487,0.071770,1.964280,0.180542,0.521064,40.615617,1.543869,0.056212,14.937402,6.233546e+03,1.107555e+05,170.438341,8.628233,54.325110,129.489940,3.772247e+11,1.715484e+11,2.066831e+12,0.882062,2.973002,4.426012,22.297524,172.318454,95.544966
std,227.773388,760.817358,0.204890,14.923190,165.902521,1.427841e+13,1.056336e+13,2.586860e+12,2.128457e+12,227.742018,2.277734,2.351324e+13,9.807262,2.137042e+12,6.517140e+11,4.743611e+11,2.770221e+12,4.756917e+12,1.267032e+13,1.339906e+13,2.758428e+12,2.200212e+12,14.923190,164.212715,366.579936,27.814039,5.243446e+12,5.242615e+12,0.816507,7.790042e+12,2.213052,1.782076e+12,1.276890e+12,75.205910,0.206044,2.299328e+13,531.520977,63.759380,6.102122,0.398899,0.951537,1305.655585,0.503111,0.619845,0.592226,0.479991,0.402268,0.422921,0.690566,0.667644,0.691573,50.326558,24.381471,229.963722,229.024236,398.632826,0.210963,2.364852,0.955535,0.205936,754.588118,21.189552,0.164997,147.957659,1.469788e+05,1.298696e+06,7648.435570,145.839644,2479.98962

In [5]:
model_a_filter = ['ebitda_margin', 'ebitda_to_interest_expense', 'debt_ratio', 'dependence_on_net_borrowings', 'net_borrowings_to_ebitda', 'ebit_margin', 'ebitda_to_sales_revenue',
                  'borrowing_dependency', 'total_borrowings_to_ebitda', 'debt_to_net_income_ratio', 'total_assets_leverage', 'current_liabilities_ratio',
                  'quick_ratio', 'days_sales_outstanding', 'average_accounts_receivable_per_sales_turnover']

investment_data_filter = ['return_on_equity', 'return_on_invested_capital', 'current_ratio', 'non_current_debt_ratio', 'equity_ratio', 'interest_coverage_ratio',
                  'debt_to_equity_ratio', 'retention_ratio', 'earnings_per_share', 'book_value_per_share', 'price_earnings_ratio', 'price_to_book_ratio',
                  'price_cash_flow_ratio', 'enterprise_value_to_ebitda', 'net_working_capital_turnover', 'fixed_asset_turnover', 'accounts_receivable_turnover',
                  'inventory_turnover', 'accounts_payable_turnover']

model_b_filter = model_a_filter + investment_data_filter

In [6]:
def process_model(model, model_filter, model_name):
    model_processed = model.copy()

    indices_to_drop = set()
    causes = {}

    for col in model_filter:
        sorted_col = model_processed[col].sort_values()

        top3_indices = sorted_col[-3:].index.tolist()
        bottom3_indices = sorted_col[:3].index.tolist() if sorted_col[:3].min() < 0 else []

        for idx in top3_indices + bottom3_indices:
            indices_to_drop.add(idx)
            corp_name = model_processed.loc[idx, 'corp']  # Get the 'corp' name
            if corp_name not in causes:
                causes[corp_name] = []
            causes[corp_name].append(col)


    model_processed = model_processed.drop(indices_to_drop)

    model_processed.reset_index(drop=True, inplace=True)

    model_with_y = model_processed[model_processed['rank'].notna()].reset_index(drop=True)
    model_without_y = model_processed[model_processed['rank'].isna()].reset_index(drop=True)

    print(f"{model_name} original shape: {model.shape}")
    print(f"{model_name} processed shape: {model_processed.shape}")
    print(f"{model_name} with y shape: {model_with_y.shape}")
    print(f"{model_name} without y shape: {model_without_y.shape}")
    print()
    
    return model_processed, model_with_y, model_without_y, causes

model_a_processed, model_a_with_y, model_a_without_y, causes_a = process_model(model_a, model_a_filter, 'model_a')
model_b_processed, model_b_with_y, model_b_without_y, causes_b = process_model(model_b, model_b_filter, 'model_b')

causes_a = str(causes_a).replace('\n', '').replace('  ', ' ')
causes_b = str(causes_b).replace('\n', '').replace('  ', ' ')


model_a original shape: (3097, 57)
model_a processed shape: (3055, 57)
model_a with y shape: (827, 57)
model_a without y shape: (2228, 57)

model_b original shape: (3097, 84)
model_b processed shape: (3009, 84)
model_b with y shape: (815, 84)
model_b without y shape: (2194, 84)



### 극단값/이상치로 인해 지워진 Row 데이터 확인

In [7]:
causes_a

"{'더메디팜': ['ebitda_margin', 'ebit_margin', 'ebitda_to_sales_revenue'], '한진칼': ['ebitda_margin', 'ebit_margin', 'ebitda_to_sales_revenue', 'days_sales_outstanding', 'average_accounts_receivable_per_sales_turnover'], '미래아이앤지': ['ebitda_margin', 'ebit_margin', 'ebitda_to_sales_revenue', 'quick_ratio'], '에스케이바이오팜': ['ebitda_margin', 'ebitda_margin', 'ebitda_to_interest_expense', 'ebit_margin', 'ebit_margin', 'ebitda_to_sales_revenue', 'ebitda_to_sales_revenue'], '롯데관광개발': ['ebitda_margin', 'ebit_margin', 'ebitda_to_sales_revenue', 'current_liabilities_ratio'], '현대퓨처넷': ['ebitda_to_interest_expense', 'ebitda_to_interest_expense'], 'LX홀딩스': ['ebitda_to_interest_expense'], '세우글로벌': ['ebitda_to_interest_expense'], 'HDC랩스': ['ebitda_to_interest_expense'], '아센디오': ['debt_ratio'], 'HJ중공업': ['debt_ratio'], '한컴라이프케어': ['debt_ratio'], 'LG헬로비전': ['dependence_on_net_borrowings', 'dependence_on_net_borrowings', 'borrowing_dependency', 'borrowing_dependency', 'total_assets_leverage', 'total_assets_lever

In [8]:
causes_b

"{'더메디팜': ['ebitda_margin', 'ebit_margin', 'ebitda_to_sales_revenue'], '한진칼': ['ebitda_margin', 'ebit_margin', 'ebitda_to_sales_revenue', 'days_sales_outstanding', 'average_accounts_receivable_per_sales_turnover'], '미래아이앤지': ['ebitda_margin', 'ebit_margin', 'ebitda_to_sales_revenue', 'quick_ratio'], '에스케이바이오팜': ['ebitda_margin', 'ebitda_margin', 'ebitda_to_interest_expense', 'ebit_margin', 'ebit_margin', 'ebitda_to_sales_revenue', 'ebitda_to_sales_revenue', 'return_on_invested_capital', 'return_on_invested_capital', 'interest_coverage_ratio', 'earnings_per_share', 'earnings_per_share', 'book_value_per_share'], '롯데관광개발': ['ebitda_margin', 'ebit_margin', 'ebitda_to_sales_revenue', 'current_liabilities_ratio'], '현대퓨처넷': ['ebitda_to_interest_expense', 'ebitda_to_interest_expense', 'interest_coverage_ratio', 'interest_coverage_ratio'], 'LX홀딩스': ['ebitda_to_interest_expense', 'equity_ratio', 'equity_ratio', 'interest_coverage_ratio'], '세우글로벌': ['ebitda_to_interest_expense', 'interest_coverag

### 이상치 처리 후 model.describe()로 비율 봤을 때 비교적 많이 좋아짐 

In [10]:
model_a_processed.describe()

,ebitda_margin,ebitda_to_interest_expense,debt_ratio,dependence_on_net_borrowings,net_borrowings_to_ebitda,revenue,cogs,selling_general_administrative_expenses,ebit,ebit_margin,ebitda_to_sales_revenue,total_assets,return_on_assets,ebitda,financial_expenses,corporate_tax,operating_cash_flow,free_cash_flow,total_liabilities,total_equity,total_borrowings,net_borrowings,borrowing_dependency,total_borrowings_to_ebitda,debt_to_net_income_ratio,total_assets_leverage,current_liabilities,working_capital,current_liabilities_ratio,quick_assets,quick_ratio,cash_and_cash_equivalents,short_term_borrowings,days_sales_outstanding,average_accounts_receivable_per_sales_turnover,market_capitalization,minimum_wag,uskor_exchange_average,PPI_year,kor_usa_ir_diff,kr_standard_yield,count,rating,paywellfare,worklifebal,culture,opportunity,manager,recommend,ceo,potential,crb_index_avg
count,3055.000000,3055.000000,3055.000000,3055.000000,3055.000000,3.055000e+03,3.055000e+03,3.055000e+03,3.055000e+03,3055.000000,3055.000000,3.055000e+03,3055.000000,3.055000e+03,3.055000e+03,3.055000e+03,3.055000e+03,3.055000e+03,3.055000e+03,3.055000e+03,3.055000e+03,3.055000e+03,3055.000000,3055.000000,3055.000000,3055.000000,3.055000e+03,3.055000e+03,3055.000000,3.055000e+03,3055.000000,3.055000e+03,3.055000e+03,3055.000000,3055.000000,3.055000e+03,3055.000000,3055.000000,3055.000000,3055.000000,3055.000000,3055.000000,3055.000000,3055.000000,3055.000000,3055.000000,3055.000000,3055.000000,3055.000000,3055.000000,3055.000000,3055.000000
mean,1.594179,22.356657,0.476859,0.449167,2.208888,3.618171e+12,2.810236e+12,4.261031e+11,2.185526e+11,1.470508,0.015942,5.312890e+12,1.492732,2.245609e+11,1.069438e+11,6.278108e+10,3.450286e+11,6.708827e+11,2.826412e+12,2.486263e+12,6.360980e+11,2.643538e+11,0.449167,4.340980,20.426900,2.529520,1.430697e+12,5.327575e+11,0.833095,1.531334e+12,1.493145,3.717442e+11,3.174609e+11,62.423742,0.171024,2.780280e+12,8487.230769,1178.166056,107.806288,0.047411,1.553682,696.981715,2.827457,2.786498,3.009133,2.595931,2.680831,2.240930,1.292455,1.318282,1.189103,214.849796
std,25.987586,208.123931,0.201857,2.080300,33.330702,1.430367e+13,1.055059e+13,2.602619e+12,2.142795e+12,25.730093,0.259876,2.354761e+13,9.125339,2.151430e+12,6.477279e+11,4.770874e+11,2.783794e+12,4.776080e+12,1.266573e+13,1.345881e+13,2.668519e+12,2.134534e+12,2.080300,41.881554,182.256418,6.327475,5.237905e+12,5.278011e+12,0.617316,7.822809e+12,1.926059,1.785283e+12,1.280839e+12,49.569976,0.135808,2.314717e+13,531.876111,63.850450,6.108159,0.399019,0.952081,1312.734037,0.503908,0.619732,0.593460,0.481451,0.403234,0.424037,0.690695,0.667804,0.691375,50.348914
min,-462.753943,-164.841190,0.014647,-3.583935,-351.630650,1.747813e+08,0.000000e+00,5.417029e+06,-3.384362e+13,-462.753943,-4.627539,1.247618e+08,-164.393583,-3.384362e+13,-7.323143e+11,-4.695683e+11,-2.347750e+13,-1.360712e+13,7.506294e+07,-9.905259e+10,0.000000e+00,-4.097272e+13,-3.583935,-391.328269,-1755.126322,-18.726740,1.752545e+08,-1.476803e+13,0.029893,7.245676e+07,0.087273,4.183128e+06,0.000000e+00,0.000000,0.000000,0.000000e+00,7530.000000,1100.580000,103.030000,-0.560000,0.500000,1.000000,0.060000,0.030000,0.030000,0.090000,0.090000,0.060000,0.060000,0.060000,0.060000,157.422309
25%,0.291772,0.004225,0.321501,0.047300,-1.436967,2.326491e+11,1.486385e+11,2.193623e+10,1.095569e+09,0.289243,0.002918,3.131309e+11,-0.095347,1.095569e+09,2.202085e+09,7.654239e+08,3.115572e+09,8.770187e+09,1.127662e+11,1.588237e+11,1.302945e+10,-2.407428e+10,0.047300,0.000000,-0.500275,1.471473,8.206505e+10,-5.502856e+09,0.488569,9.519346e+10,0.649499,1.775525e+10,2.977139e+09,39.969566,0.109506,1.134717e+11,8350.000000,1144.610000,103.480000,-0.290000,1.000000,129.000000,2.500000,2.360000,2.640000,2.260000,2.440000,1.970000,0.760000,0.815000,0.660000,187.793544
50%,3.748585,2.277636,0.486289,0.222276,0.114441,6.454121e+11,4.713924e+11,6.085410e+10,2.085844e+10,3.744591,0.037486,7.604555e+11,2.332586,2.087800e+10,8.298356e+09,4.866

In [11]:
model_b_processed.describe()

,ebitda_margin,ebitda_to_interest_expense,debt_ratio,dependence_on_net_borrowings,net_borrowings_to_ebitda,revenue,cogs,selling_general_administrative_expenses,ebit,ebit_margin,ebitda_to_sales_revenue,total_assets,return_on_assets,ebitda,financial_expenses,corporate_tax,operating_cash_flow,free_cash_flow,total_liabilities,total_equity,total_borrowings,net_borrowings,borrowing_dependency,total_borrowings_to_ebitda,debt_to_net_income_ratio,total_assets_leverage,current_liabilities,working_capital,current_liabilities_ratio,quick_assets,quick_ratio,cash_and_cash_equivalents,short_term_borrowings,days_sales_outstanding,average_accounts_receivable_per_sales_turnover,market_capitalization,minimum_wag,uskor_exchange_average,PPI_year,kor_usa_ir_diff,kr_standard_yield,count,rating,paywellfare,worklifebal,culture,opportunity,manager,recommend,ceo,potential,crb_index_avg,gross_profit_margin,operating_profit_margin,net_profit_margin,return_on_equity,return_on_invested_capital,current_ratio,non_current_debt_ratio,equity_ratio,interest_coverage_ratio,debt_to_equity_ratio,net_debt_ratio,retention_ratio,earnings_per_share,book_value_per_share,price_earnings_ratio,price_to_book_ratio,price_cash_flow_ratio,enterprise_value_to_ebitda,operating_income,net_income,fixed_assets,total_asset_turnover,net_working_capital_turnover,fixed_asset_turnover,accounts_receivable_turnover,inventory_turnover,accounts_payable_turnover
count,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3.009000e+03,3.009000e+03,3.009000e+03,3.009000e+03,3009.000000,3009.000000,3.009000e+03,3009.000000,3.009000e+03,3.009000e+03,3.009000e+03,3.009000e+03,3.009000e+03,3.009000e+03,3.009000e+03,3.009000e+03,3.009000e+03,3009.000000,3009.000000,3009.000000,3009.000000,3.009000e+03,3.009000e+03,3009.000000,3.009000e+03,3009.000000,3.009000e+03,3.009000e+03,3009.000000,3009.000000,3.009000e+03,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3.009000e+03,3.009000e+03,3009.000000,3009.000000,3009.000000,3009.000000,3.009000e+03,3.009000e+03,3.009000e+03,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000,3009.000000
mean,1.739089,19.485898,0.475152,0.443615,2.205938,3.647568e+12,2.832195e+12,4.296418e+11,2.205529e+11,1.614377,0.017391,5.345989e+12,1.617569,2.266508e+11,1.079907e+11,6.366386e+10,3.509810e+11,6.807344e+11,2.832379e+12,2.513393e+12,6.329950e+11,2.577666e+11,0.443615,4.274441,17.968492,2.502306,1.421322e+12,5.399825e+11,0.833001,1.524863e+12,1.483236,3.752284e+11,3.130172e+11,62.598375,0.171502,2.771127e+12,8489.471585,1178.413144,107.832154,0.048611,1.556165,699.109658,2.830083,2.790196,3.009684,2.597139,2.682738,2.242732,1.294510,1.320156,1.190808,215.035761,23.009491,9.215623,0.831175,-0.357440,0.069818,1.920986,0.146461,0.524697,19.342844,1.455280,0.054902,11.935615,4.903211e+03,8.158098e+04,18.025637,2.648109,30.741752,14.688468,3.835042e+11,1.765967e+11,2.094901e+12,0.883267,3.783297,4.220118,11.638691,87.775684,27.930581
std,25.340286,132.937711,0.200250,2.090117,33.440247,1.439978e+13,1.062022e+13,2.620781e+12,2.158739e+12,25.074958,0.253403,2.369641e+13,8.473978,2.167436e+12,6.523629e+11,4.805825e+11,2.803585e+12,4.811019e+12,1.272195e+13,1.355761e+13,2.664062e+12,2.125298e+12,2.090117,42.019800,160.743357,6.235536,5.194202e+12,5.317849e+12,0.619804,7.828157e+12,1.837950,1.798125e+12,1.251238e+12,49.742654,0.136281,2.323615e+13,531.561348,63.953090,6.119840,0.398750,0.954219,1319.200334,0.504317,0.619433,0.593810,0.482177,0.403178,0.425220,0.689199,0.666291,0.689751,50.453050,22.595751,26.348070,27.861918,42.861385,0.186340,2.009905,0.115460,0.200475,131.366661,3.899491,0.161270,82.205282,5.141602e+04,3.578535e+05,284.565690,27.776177,675.219192,356.477965,2.470312